In [41]:
import pyodbc
import sys
import logging
from datetime import datetime

In [42]:
logging.basicConfig(
    filename="hotel.log",
    level=logging.DEBUG,
    format="%(username)s - %(asctime)s - %(levelname)s - %(message)s",
    datefmt="%d-%m-%Y %H:%M:%S",
    filemode="a",
    force=True
)    


In [ ]:
class Dwarka:
    def __init__(self):
        self.order_dict = {}
        self.total_bill = 0
        self.current_user = "ROOT"
    def DatabaseCreation(self):
        self.logger = logging.LoggerAdapter(
                    logging.getLogger(),
                    {"username": self.current_user}
                        )            


        
        try:

            self.conn = pyodbc.connect(
                r'Driver=ODBC Driver 18 for SQL Server;'
                r'Server=SEJAL\SQLEXPRESS;'
                r'Database=Bill_DB;'
                r'Trusted_Connection=yes;'
                r'Encrypt=no;'
            )

            
            self.have_account=input("Do you have Dwarka restrarunt account(y/n):").lower()

            
            self.login()
            self.logger.info(" logged in  successfully")
            
            
        except:
            self.logger.error("Connection failed")
            print(sys.exc_info())

    def login(self):
        self.cursor=self.conn.cursor()
        
        if self.have_account=='y':
            self.username=input("Please enter the username:")
            self.password=input("Please enter the password:")
            self.cursor.execute("""
            SELECT * FROM Users
            WHERE Username = ? AND Password = ?
            """, (self.username, self.password))

            user = self.cursor.fetchone()
            
            if user:
                print("Login Successful")
                print("---------------------")
                print("You may order the food")
                self.user_id = user[0]
                self.current_user = self.username
                self.logger = logging.LoggerAdapter(
                logging.getLogger(),
                {"username": self.current_user}
                )
                self.TakeOrder()
            else:
                print("Invalid username or password")

        elif self.have_account=='n':
            print("Please signup to order food!!")
            self.SignUp()

        else:
            print("Invalid login data!")

    def SignUp(self):
        self.enter_username=input("Enter your username:")
        self.enter_password=input("Enter a password:")

        try:
            cursor=self.conn.cursor()
            cursor.execute("""Insert into Users(Username,Password)values (?,?)""",(self.enter_username,self.enter_password))
            self.conn.commit()
            print("Account created successfully")
            print("------------------")
            print("Login now")
            self.logger.debug("You are in SignIn function now")
            self.login()


        except Exception as e:
            print("Actual error:", e)

                                


    def TakeOrder(self):
        self.logger.debug("You are in TakeOrder function now")
        
        while True:
            try:
                print(f"Name of customer{self.username}")
                
                self.logger.info("Customer is ordering a food")
                str1 = input("Do you want to order some food!!(Y/N)").lower()
                print("Welcome Hotel Dwarka")
                print("--------------------------")
                if str1 == 'y' :
                    print("Please select menu you want to order!")
                    cursor=self.conn.cursor()
                    cursor.execute('SELECT * FROM dbo.MenuCard')

                    records=cursor.fetchall()
                    for row in records:
                        print(row)
                    while True:
                            self.item=input("Enter a menu item you want or enter done:").capitalize()

                            if self.item.lower() == 'done':
                                self.PlaceOrder()
                                self.GenerateBill()
                                self.PrintBill()
                                break

                            
                            for row in records:
                                if self.item==row.ItemName :
                                    self.qty=int(input("How much quantity:"))
                                    price = row.Price
                                    self.item_total = self.qty * price
                                    self.total_bill += self.item_total  


                                    self.order_dict[self.item] = {
                                    "qty": self.qty,
                                    "price": price,
                                    "item_total":self.item_total
                                    }
                                    
                        
                                    break
                              
                                
                               
                else:
                    print("Thank you!")
                break
                   
                

            except ValueError:
                self.logger.error("Entering invalid data")
                print("Please enter correct data!!")


    def PlaceOrder(self):
        
        cursor=self.conn.cursor()
        self.logger.debug("Cursor connection is created")
        for item, details in self.order_dict.items():
                cursor.execute("""
                INSERT INTO Orders (ItemName, Quantity, Price, ItemTotal)
                VALUES (?, ?, ?, ?)
            """,
            item,
            details["qty"],
            details["price"],
            details["item_total"]
            )

        self.conn.commit()
        self.logger.debug(f"order saved successfully {self.order_dict.items()}")
        print("Order saved successfully!")
        

    def GenerateBill(self):
        self.logger.info("You are in GenerateBill function")
        today_date = datetime.now().strftime("%d-%m-%Y")

        print("-----------------------------------------------------------------------------------")
        print("{:^60}".format("Welcome Hotel Dwarka\n"))
        print("-----------------------------------------------------------------------------------")
        print(f"User ID       : {self.user_id}")
        print(f"Customer Name : {self.current_user}")
        print(f"Date          : {today_date}")
        print("-----------------------------------------------------------------------------------")
        print("{:<10}{:<20}{:<10}{:<10}{:>10}".format("SR","Menu","Qty","Prc","Total"))
        print("-----------------------------------------------------------------------------------")

        sr = 1
        for item, details in self.order_dict.items():
            print("{:<10}{:<20}{:<10}{:<10.2f}{:>10.2f}".format(
                sr,
                item,
                details["qty"],
                details["price"],
                details["item_total"]
            ))
            sr += 1

        print("-----------------------------------------------------------------------------------")
        print("{:<50}{:>9.2f}".format("Grand Total", self.total_bill))
        print("-----------------------------------------------------------------------------------")
        self.logger.debug(f"Final Grand Total Calculated: {self.total_bill}")



    def PrintBill(self):
         today_date = datetime.now().strftime("%d-%m-%Y")


         self.filename=r"F:\Bizmetric\Training\Python\bill.txt"
         with open(self.filename, "w") as f:

            f.write("------------------------------------------------------------\n")
            f.write("                 Welcome Hotel Dwarka\n")
            f.write("------------------------------------------------------------\n")
            f.write(f"User ID       : {self.user_id}\n")
            f.write(f"Customer Name : {self.current_user}\n")
            f.write(f"Date          : {today_date}\n")
            f.write("------------------------------------------------------------\n")
            f.write("{:<5}{:<20}{:<10}{:<10}{:<10}\n".format(
                "SR", "Menu", "Qty", "Price", "Total"))
            f.write("------------------------------------------------------------\n")

            sr = 1
            for item, details in self.order_dict.items():
                f.write("{:<5}{:<20}{:<10}{:<10.2f}{:<10.2f}\n".format(
                    sr,
                    item,
                    details["qty"],
                    details["price"],
                    details["item_total"]
                ))
                sr += 1

            f.write("------------------------------------------------------------\n")
            f.write("{:<45}{:<10.2f}\n".format("Grand Total", self.total_bill))
            f.write("------------------------------------------------------------\n")

            print(f"Bill saved successfully as {self.filename}")
            print(f"Final Grand Total Calculated: {self.total_bill}")
            self.logger.debug(f"Final Grand Total Calculated: {self.total_bill}")
            self.logger.info("Bill is printed succesfully")

    def SignOut(self):
        try:
            self.sign_out=input("Do you want to signout(y/n):").lower()
            if self.sign_out=='y':
                self.cursor.close()
                print("You are logout successfully!!")

            else:
                print("Error occured during signout!")


        except Exception as e:
            print("Error is:",e)
        


        


In [48]:
d=Dwarka()
d.DatabaseCreation()

Login Successful
---------------------
You may order the food
Name of customersejal
Welcome Hotel Dwarka
--------------------------
Please select menu you want to order!
(1, 'Idli', 'Breakfast', Decimal('30.00'))
(2, 'Dosa', 'Breakfast', Decimal('60.00'))
(3, 'Poha', 'Breakfast', Decimal('20.00'))
(4, 'Upma', 'Breakfast', Decimal('35.00'))
(5, 'Sandwich', 'Snacks', Decimal('60.00'))
(6, 'Tea', 'Beverage', Decimal('15.00'))
(7, 'Coffee', 'Beverage', Decimal('25.00'))
Order saved successfully!
-----------------------------------------------------------------------------------
                   Welcome Hotel Dwarka
                    
-----------------------------------------------------------------------------------
User ID       : 1
Customer Name : sejal
Date          : 19-02-2026
-----------------------------------------------------------------------------------
SR        Menu                Qty       Prc            Total
--------------------------------------------------------------